In [1]:
import dump_data
import collections
import json
import re
import os
import math
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import matplotlib
import random
import itertools

2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG $HOME=/Users/dusanklinec
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG matplotlib data path /Users/dusanklinec/.pyenv/versions/3.6.5/lib/python3.6/site-packages/matplotlib/mpl-data
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG loaded rc file /Users/dusanklinec/.pyenv/versions/3.6.5/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG matplotlib version 2.2.3
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG interactive is False
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG platform is darwin
2019-02-27 22:48:14 phx.local matplotlib[52276] DEBUG loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_w

2019-02-27 22:48:15 phx.local matplotlib[52276] DEBUG CACHEDIR=/Users/dusanklinec/.matplotlib
2019-02-27 22:48:15 phx.local matplotlib.font_manager[52276] DEBUG Using fontManager instance from /Users/dusanklinec/.matplotlib/fontList.json
2019-02-27 22:48:15 phx.local matplotlib.backends[52276] DEBUG backend module://ipykernel.pylab.backend_inline version unknown
2019-02-27 22:48:16 phx.local matplotlib.backends[52276] DEBUG backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
loader = dump_data.Loader()

In [ ]:
loader.load({'no_pvals':True, 'only_pval_cnt': True})

2019-02-27 22:48:22 phx.local dump_data[52276] DEBUG Unparsed: ['-f', '/Users/dusanklinec/Library/Jupyter/runtime/kernel-7a2452be-1af4-40ed-afd0-44991f4169f3.json']
2019-02-27 22:48:22 phx.local dump_data[52276] INFO Loading all experiments
2019-02-27 22:48:23 phx.local dump_data[52276] INFO Number of all experiments: 1326
2019-02-27 22:48:23 phx.local dump_data[52276] INFO Loading all batteries, len: 1326
2019-02-27 22:48:29 phx.local dump_data[52276] INFO Loading all tests, len: 9281
2019-02-27 22:48:29 phx.local dump_data[52276] INFO Loading all variants params, len: 1000
2019-02-27 22:48:30 phx.local dump_data[52276] INFO Loading all variant params, len: 2000
2019-02-27 22:48:31 phx.local dump_data[52276] INFO Loading all variants params, len: 1000
2019-02-27 22:48:31 phx.local dump_data[52276] INFO Loading all variant params, len: 2000
2019-02-27 22:48:33 phx.local dump_data[52276] INFO Loading all variants params, len: 1000
2019-02-27 22:48:33 phx.local dump_data[52276] INFO Load

In [ ]:
loader.process()

In [ ]:
loader.experiments

## Test categorization

In [6]:
# Data sizes -> tests -> test_config -> counts
test_configs = collections.defaultdict(
    lambda: collections.defaultdict(
        lambda: collections.defaultdict(
            lambda: 0
        )))

# Data sizes -> tests -> test_config -> config_data -> counts
test_configs_val = collections.defaultdict(
    lambda: collections.defaultdict(
        lambda: collections.defaultdict(
            lambda: collections.defaultdict(
                lambda: 0
        ))))


# data size -> test_flat -> counts
flat_configs_types = collections.defaultdict(
    lambda: collections.defaultdict(
            lambda: 0
        ))

# data size -> test_flat -> counts
flat_configs = collections.defaultdict(
    lambda: collections.defaultdict(
            lambda: 0
        ))

# Data sizes -> tests -> test_config+variantval -> counts
test_configs_var = collections.defaultdict(
    lambda: collections.defaultdict(
        lambda: collections.defaultdict(
            lambda: 0
        )))


for tt in loader.tests.values():
    exp = tt.battery.exp
    size = exp.exp_info.size
    tt_id = '|'.join(reversed(tt.short_desc()))

    for vv in tt.variants.values():
        cfs = '|'.join([str(x) for x in vv.settings.keys_tuple()])
        cfv = '|'.join([str(x) for x in vv.settings.values_tuple()])

        for ss in vv.sub_tests.values():
            tfs = '|'.join([str(x) for x in ss.params.keys_tuple()])
            tfv = '|'.join([str(x) for x in ss.params.values_tuple()])
            tcfg = '{%s}{%s}' % (cfs, tfs)
            tcfg_val = '{%s}{%s}' % (cfv, tfv)
            type_val = '[%s][%s]' % (tt_id, tcfg)
            var_val = '{%s}{%s}' % (cfs, cfv) 
            full_val = '[%s][%s][%s]' % (tt_id, tcfg, tcfg_val)

            test_configs[size][tt_id][tcfg] += 1
            test_configs['ALL'][tt_id][tcfg] += 1
            
            test_configs_var[size][tt_id][var_val] += 1
            test_configs_var['ALL'][tt_id][var_val] += 1
            
            test_configs_val[size][tt_id][tcfg][tcfg_val] += 1
            test_configs_val['ALL'][tt_id][tcfg][tcfg_val] += 1
            
            flat_configs[size][full_val] += 1
            flat_configs['ALL'][full_val] += 1
            flat_configs_types[size][type_val] += 1
            flat_configs_types['ALL'][type_val] += 1
            
            

In [7]:
print(json.dumps(test_configs_var, indent=2))

{
  "8589934592": {
    "TestU01 Block Alphabit|smultin_MultinomialBitsOver": {
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|1|1}": 1768,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|2|1}": 1768,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|4|1}": 1768,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|8|1}": 1768,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|16|1}": 1768,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|32|1}": 1768
    },
    "TestU01 Block Alphabit|sstring_HammingIndep": {
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|1|1}": 884,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|2|1}": 884,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|4|1}": 884,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|8|1}": 884,
      "{Bit NB|Bit R|Bit S|Bit W|Repetitions}{8388608000|0|32|16|1}": 884,
      "{Bit NB|Bit R|Bit S|Bit W|Repet

In [8]:
stables = set()
for msize in flat_configs_types:
    tests = flat_configs_types[msize]
    for t in tests:
        stables.add(t)
    
for msize in flat_configs_types:
    tests = flat_configs_types[msize]
    for test in tests:
        # other sizes
        for msize_other in [x for x in flat_configs_types if x != msize]:
            if test not in flat_configs_types[msize_other]:
                stables.discard(test)
                print('DIf: %10d->%10d: %s' % (msize, msize_other, test))
    #print(msize)
    
print('Stables....')
for xx in stables:
    printx('.. %s' % xx)

DIf: 8589934592->  10485760: [Dieharder|Marsaglia and Tsang GCD Test][{P-sample count}{}]
DIf: 8589934592->  10485760: [Dieharder|Byte Distribution][{P-sample count}{}]
DIf: 8589934592-> 104857600: [Dieharder|Byte Distribution][{P-sample count}{}]
DIf: 8589934592->  10485760: [Dieharder|DAB DCT][{P-sample count}{}]
DIf: 8589934592->  10485760: [Dieharder|DAB Fill Tree Test][{P-sample count}{}]
DIf: 8589934592-> 104857600: [Dieharder|DAB Fill Tree Test][{P-sample count}{}]
DIf: 8589934592->  10485760: [Dieharder|DAB Fill Tree Test 2][{P-sample count}{}]
DIf: 8589934592-> 104857600: [Dieharder|DAB Fill Tree Test 2][{P-sample count}{}]
DIf: 8589934592->  10485760: [Dieharder|DAB Monobit 2 Test][{P-sample count}{}]
DIf: 8589934592-> 104857600: [Dieharder|DAB Monobit 2 Test][{P-sample count}{}]
DIf: 8589934592->  10485760: [TestU01 Small Crush|smarsa_BirthdaySpacings][{Repetitions}{N|d|n|p|r|t}]
DIf: 8589934592->  10485760: [TestU01 Small Crush|sknuth_Collision][{Repetitions}{N|Sparse|d|n|r

TypeError: %d format: a number is required, not str

## Test analysis

In [ ]:
# Test analysis and scoring
exps = list(loader.experiments.values())
exps.sort(key=lambda x: (x.exp_info.size, x.exp_info.meth, x.exp_info.seed, x.exp_info.fnc))
rev_exp = {x.name: i for i,x in enumerate(exps)}

In [ ]:
# Iterate over test
# test[name] = [pvals] per experiment, on a fixed position. None if not present.
tests = collections.defaultdict(lambda: [None] * len(exps))
for tt in loader.tests.values():
    tt_id = '|'.join(reversed(tt.short_desc()))
    tests[tt_id][rev_exp[tt.battery.exp.name]] = tt.get_single_pval()
    # print(tt.get_single_pval(), tt.shidak_alpha(0.10), tt.summarized_pvals)
    
# Sort tests, so we have defined ordering
tests_srt = [(k,tests[k]) for k in tests]
tests_srt.sort(key=lambda x: x[0])

In [ ]:
def median(inp, is_sorted=False):
    inp = sorted(inp) if not is_sorted else inp
    return None if not inp else (inp[len(inp)//2])

def filter_experiments(exps, fnc):
    return [bool(fnc(x)) for x in exps]

def project_tests(tests_srt, bitmap):
    return [(x[0], list(itertools.compress(x[1], bitmap))) for x in tests_srt]
    
def get_rounds(exps):
    f2r = collections.defaultdict(lambda: [])
    for x in exps:
        f2r[x.exp_info.fnc_name].append(x.exp_info.fnc_round)
    return f2r
    
def get_top_rounds(exps):
    f2r = get_rounds(exps)
    return {k: max(f2r[k]) for k in f2r}

def get_low_rounds(exps):
    f2r = get_rounds(exps)
    return {k: min(f2r[k]) for k in f2r}

def get_med_rounds(exps):
    f2r = get_rounds(exps)
    return {k: median(f2r[k]) for k in f2r}

def get_exid(rev_exp, bitmap, name):  # name -> new index
    idx = rev_exp[name]
    return sum(bitmap[:idx])

def get_ex_byidx(bitmap, idx):  # new index -> old index
    oidx = 0
    cnt = 0
    for i, x in enumerate(bitmap):
        if x == 1:  # selected to new round
            if cnt == idx:
                return oidx
            cnt += 1
        oidx += 1
    raise ValueError('Not found')
    
def get_maximum_detections(selection, tests_srt, do_p):
    ctests = project_tests(tests_srt, selection)
    total_det = sum(selection) * len(tests_srt)
    tests_undefined = collections.defaultdict(lambda: 0)  # tname -> # of NONE in test
    total_def_det = 0

    # Fails removal & report
    for tname, tvals in ctests:
        for idx, tval in enumerate(tvals):
            if tval is None:
                eidx = get_ex_byidx(selection, idx)
                tests_undefined[tname] += 1
                #print('%s : %s' % (tname, exps[eidx].name))
            else:
                total_def_det += 1
    return total_def_det     
    
def get_detections(ctests, alpha):
    totals = len(ctests) * len(ctests[0][1])
    test_fails = [sum(1 for y in x[1] if y is None) for x in ctests]
    tests_detections = [(x[0], 
                         sum(1 for y in x[1] if y is not None and y <= alpha) / (len(x[1]) - test_fails[i])) 
                        for i, x in enumerate(ctests) if (len(x[1]) - test_fails[i]) > 0]
    return tests_detections
    

In [ ]:
top_rounds = get_top_rounds(exps)
low_rounds = get_low_rounds(exps)
med_rounds = get_med_rounds(exps)
tf_10mib = filter_experiments(exps, lambda x: x.exp_info.osize=='10MiB')
tf_100mib = filter_experiments(exps, lambda x: x.exp_info.osize=='100MiB')
tf_8gib = filter_experiments(exps, lambda x: x.exp_info.osize=='8GiB')
tf_topr = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round))
tf_topr8 = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and x.exp_info.osize=='8GiB')
tf_topr8_ctr = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and x.exp_info.osize=='8GiB' and x.exp_info.meth == 'ctr')
tf_topr8_aes = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.osize=='8GiB')
tf_lowr8_aes = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.osize=='8GiB')
tf_topr8_aes_ctr = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.meth == 'ctr' and x.exp_info.osize=='8GiB')
tf_lowr8_aes_ctr = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.meth == 'ctr' and x.exp_info.osize=='8GiB')
tf_topr8_aes_hw = filter_experiments(exps, lambda x: (top_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.meth == 'hw' and x.exp_info.osize=='8GiB')
tf_lowr8_aes_hw = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and 'AES' in x.exp_info.fnc_name and x.exp_info.meth == 'hw' and x.exp_info.osize=='8GiB')
tf_lowr = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round))
tf_lowr8 = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and x.exp_info.osize=='8GiB')
tf_lowr8_ctr = filter_experiments(exps, lambda x: (low_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and x.exp_info.osize=='8GiB')
tf_medr8 = filter_experiments(exps, lambda x: (med_rounds[x.exp_info.fnc_name] == x.exp_info.fnc_round) and x.exp_info.osize=='8GiB' and x.exp_info.meth == 'ctr')

In [ ]:
#tests_detections = [(x[0], sum(1 for y in x[1] if y is not None and y <= 0.01)) for x in project_tests(tests_srt, tf_lowr)]

tests_detections = get_detections(project_tests(tests_srt, tf_lowr8_aes_hw), 0.01)
fig, ax = pyplot.subplots(figsize=(6*3, 2*3))
ax.tick_params(axis='x', labelrotation=90)
plt.scatter([x[0] for x in tests_detections], 
            [x[1] for x in tests_detections], 
            label='plot', s=10)
plt.legend(loc='best')
ax.set_ylabel("relative detections (H0 rejections)")

In [ ]:
# TODO: filter top 3 strongest and top 3 lowest tests, 10, 100 and 8 GB

In [ ]:
step =0.0001
alpha=0.0001 - step

selection = tf_lowr8_aes_hw

ctests = project_tests(tests_srt, selection)
total_det = sum(selection) * len(tests_srt)
tests_undefined = collections.defaultdict(lambda: 0)  # tname -> # of NONE in test
total_def_det = 0

# Fails removal & report
for tname, tvals in ctests:
    for idx, tval in enumerate(tvals):
        if tval is None:
            eidx = get_ex_byidx(selection, idx)
            tests_undefined[tname] += 1
            #print('%s : %s' % (tname, exps[eidx].name))
        else:
            total_def_det += 1
            
print('Total detections: %s, defined: %s' % (total_det, total_def_det))


x_data = []
y_data = []
while alpha <= 1.0:
    alpha += step
    
    tests_detections = [(x[0], sum(1 for y in x[1] if y is not None and y <= alpha)) for x in ctests]
    detections = sum([x[1] for x in tests_detections])
    
    if min([abs(alpha - x) for x in [0.005, 0.001, 0.01, 0.05, 0.1]]) <= 0.000001:
        print('Alpha: %0.8f, detections: %s, relative: %s' % (alpha, detections, float(detections)/total_def_det))
        
    x_data.append(alpha)
    #y_data.append(float(detections))
    #y_data.append(alpha / (float(detections)/total_def_det))
    #y_data.append((float(detections)/total_def_det) / alpha)
    y_data.append((float(detections)/total_def_det))
    
fig, ax = pyplot.subplots(figsize=(6*2, 4*2))
plt.scatter(x_data, 
            y_data, 
            label='plot', s=4)
plt.plot(x_data, x_data, label='y=x', color='red')
plt.legend(loc='best')  # plt.tick_params(labelsize=14)
ax.set_xlabel("alpha")
ax.set_ylabel("relative detections (H0 rejections)")

## Vertical scoring

In [ ]:
# Idea: work in the test matrix in columns, count number of test reports per data set and assign points.
# Tests ranking results together.
# T1: per data: score tests. N points, divide between activated tests
# T2: per data: add tests to the set. if set exists, increment number. Which are the most often sets?
# T3: per data: triangular matrix of N tests. add +1 to a test which fired together. Which are the most firing tests?

In [ ]:
selection = tf_lowr8  # tf_8gib
alpha = 0.01 

tests_scoring1 = collections.defaultdict(lambda: 0)
tests_scoring2 = collections.defaultdict(lambda: 0)
tests_scoring_eq = collections.defaultdict(lambda: 0)
tests_sets = collections.defaultdict(lambda: 0)
tests_matrix = [[0]*len(tests_srt) for i in range(len(tests_srt))]
tests_matrix_2 = [[0]*len(tests_srt) for i in range(len(tests_srt))]

ctests = project_tests(tests_srt, selection)
total_det = sum(selection) * len(tests_srt)
tests_undefined = collections.defaultdict(lambda: 0) 
num_tests = len(ctests)

for col in range(len(ctests[0][1])):
    num_defined = sum(1 for i in range(num_tests) if ctests[i][1][col] is not None)
    detections = [i for i in range(num_tests) if ctests[i][1][col] is not None and ctests[i][1][col] < alpha]
    num_detections = len(detections)
    
    # T1
    points_to_add1 = num_defined // num_detections if num_detections > 0 else 0
    points_to_add2 = num_defined // (num_defined-num_detections) if num_defined != num_detections > 0 else 0
    for ti in range(num_tests):
        tests_scoring_eq[ti] += points_to_add1 if ti in detections else points_to_add2
        tests_scoring1[ti] += points_to_add1 if ti in detections else 0
        tests_scoring2[ti] += 0 if ti in detections else points_to_add2
        
    # T2
    tests_sets[tuple(detections)] += 1
    
    # T3
    for ti in detections:
        for ti2 in detections:
            if ti >= ti2:
                continue
            tests_matrix[ti][ti2] += 1
    
    pairs = set([(i, j) for i,j in itertools.product(detections, detections) if i<j])
    for i in range(len(ctests)):
        for j in range(i, len(ctests)):
            if (i,j) not in pairs:
                tests_matrix_2[i][j] += 1
    

In [ ]:
# T1 test uniqueness scoring
fig, ax = pyplot.subplots(figsize=(6*3, 2*3))
ax.tick_params(axis='x', labelrotation=90)

sns.barplot(x=[x[0] for x in ctests], y=[tests_scoring1[x] for x in range(len(ctests))])
ax.set_ylabel("test scoring (the better the more unique appearances)")

In [ ]:
# T2: test sets
tests_sets_l = sorted([(k, tests_sets[k]) for k in tests_sets if tests_sets[k]>=2], reverse=True)
print(tests_sets_l[:10])

In [ ]:
# T3: matrix
import networkx as nx
import matplotlib.pyplot as plt
from graphviz import Digraph, Graph

In [ ]:
print(tests_matrix_2)

In [ ]:
G = nx.Graph()
G2 = Graph('G')
for t in ctests:
    G.add_node(t[0])

for i in range(len(ctests)):
    for j in range(i+1, len(ctests)):
        w = tests_matrix_2[i][j]
        if w > 15:
            continue
        print("%s - %s [%s]" % (ctests[i][0], ctests[j][0], w))
        G.add_edge(ctests[i][0], ctests[j][0], weight=w, color='green' if w<=5 else 'black', size=10 if w<=5 else 2)
        G2.edge(str(ctests[i][0]), str(ctests[j][0]), 'weight=%s' % w)# weight=w)

#print(json.dumps([list(x) for x in nx.connected_components(G)], indent=2))

In [ ]:
G2.view()

In [ ]:
plt.figure(figsize=(24, 24))
#nx.draw_networkx(G, with_labels=True)
#nx.draw(G, with_labels=True)
nx.draw(G, with_labels=True, pos=nx.circular_layout(G))
#nx.draw(G, with_labels=True, pos=nx.kamada_kawai_layout(G))
#nx.draw(G, with_labels=True, pos=nx.shell_layout(G, shells))
#nx.draw(self.G, with_labels=True, pos=mylay(self.shells))
#plt.savefig("/tmp/deps.png", dpi=400)

In [56]:
a=[1,2,3,4,5,6]
print(list(itertools.product(a,a)))

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)]


In [ ]:
def sidak_alpha(alpha, m):
    return 1 - (1 - alpha)**(1./m)


def sidak_inv(alpha, m):
    return 1 - (1 - alpha)**m


def chunks(items, size):
    for i in range(0, len(items), size):
        yield items[i : i + size]


def merge_pvals(pvals, batch=2):
    if len(pvals) <= 1:
        return pvals
    
    batch = min(max(2, batch), len(pvals))  # norm batch size
    parts = list(chunks(pvals, batch))
    exponent = len(pvals) / len(parts)
    npvals = []
    for p in parts:
        pi = sidak_inv(min(p), exponent)
        npvals.append(pi)
    # print(npvals, [len(x) for x in parts])
    return merge_pvals(npvals, batch)
    

In [ ]:
# SIDAK TEST
alpha= 0.0085 #.015  # .005
pvals=[.03, .5, .21, ] + [0.001, 0.11, 0.99,]
nvals=[.07, .02, .002009, 0.98, 0.55,   0.0001 ]
ns_ex=sidak_alpha(alpha, len(nvals))
ns_cp=sidak_alpha(min(nvals), len(nvals))
nsicp=sidak_inv(min(nvals), len(nvals))
         
print('alpha: %s' % alpha)
print('min p: %s' % min(nvals))
print('sh  t: %s, %s' % (ns_ex, min(nvals) < ns_ex))
print('shi p: %s, %s' % (nsicp, nsicp < alpha))
print('')

t1 = pvals+nvals
a1 = sidak_alpha(alpha, len(t1))
print(min(t1), a1, min(t1) < a1, len(t1), "\n")

t2 = pvals+[sidak_inv(min(nvals), len(nvals))]
a2 = sidak_alpha(alpha, len(t2))
print(min(t2), a2, min(t2) < a2, len(t2), "\n")

dexp = (len(t1)) / (len(pvals)+1.)
Y = 1-(1-min(t1))**(dexp)
t3 = pvals+[Y]
a3 = sidak_alpha(alpha, len(t3))
print(Y)
print(min(t3), a2, min(t3) < a3, len(t3), "\n")
#print(sidak_inv(0.1, 5))
#print(sidak_inv(0.01, 5))

#random.shuffle(t1)
print('='*80)
print('LEN: %s' % len(t1))
ideal = merge_pvals(t1, 999)
print('M: ', ideal) # FULL merge
#ammend(t1)
#print('M: ', merge_pvals(t1, 999)) # FULL merge
print('M: ', merge_pvals(t1, 2))
print('M: ', merge_pvals(t1, 3))
print('M: ', merge_pvals(t1, 4))
print('M: ', merge_pvals(t1, 5))
print('M: ', merge_pvals(t1, 6))
print('M: ', merge_pvals(t1, 7))
print('M: ', merge_pvals(t1, 8))
print('M: ', merge_pvals(t1, 9))
print('M: ', merge_pvals(t1, 10))

In [ ]:
res_chars = {
    "False|Anderson-Darling (A2)|Kolmogorov-Smirnov (D+)|Kolmogorov-Smirnov (D-)|Std empirical corr|Std empirical mean": 12372,
    "False|Chi-square": 49314,
    "False|Normal": 19811,
    "False|Chi-square(C)|Chi-square(H)|Chi-square(J)|Chi-square(M)|Chi-square(R)": 25980,
    "True|Chi-Square": 247182,
    "False|Collision": 10350,
    "False|Bit distance": 3489,
    "False|Chi-square|Normal": 3042,
    "False|Anderson-Darling|Kolmogorov-Smirnov (D+)|Kolmogorov-Smirnov (D-)": 2613,
    "False|Chi-square|Global longest run of 1": 2184,
    "False|Anderson-Darling|Kolmogorov-Smirnov (D+)|Kolmogorov-Smirnov (D-)|Normal|Sample variance": 3029,
    "False|Anderson-Darling (A2) (bits)|Anderson-Darling (A2) (runs)|Chi-square (runs)|Kolmogorov-Smirnov (D+) (bits)|Kolmogorov-Smirnov (D+) (runs)|Kolmogorov-Smirnov (D-) (bits)|Kolmogorov-Smirnov (D-) (runs)|Normal (bits)|Sample variance (bits)": 442,
    "False|Anderson-Darling|Chi-square": 1724,
    "True|Kolmogorov-Smirnov": 121542,
    "False|A2 (m-NP)|A2 (mNP1)|AD (NP)|AD (mNP2)|Jumps Y (mN)": 413,
    "False|A2 (m-NP)|A2 (mNP1)|AD (NP)|AD (mNP2)|AD (mNP2-S)|Jumps Y (mN)": 824,
    "False|Anderson-Darling - AD (A2)|Anderson-Darling - KS (D+)|Anderson-Darling - KS (D-)|Chi-square - AD (A2)|Chi-square - KS (D+)|Chi-square - KS (D-)|Observations sums": 858,
    "False|Anderson-Darling": 858,
    "False|Anderson-Darling|Chi-square|Kolmogorov-Smirnov (D+)|Kolmogorov-Smirnov (D-)": 858,
    "False|Unknown 0|Unknown 1|Unknown 2|Unknown 3": 1,
    "False|Unknown 0": 26520,
    "False|Unknown 0|Unknown 1": 884
}

In [ ]:
chi=0
kolm=0
ander=0
other=0
for k in res_chars:
    kl = k.lower()
    if 'chi' in kl:
        chi+=res_chars[k]
    elif 'kolm' in kl:
        kolm+=res_chars[k]
    elif 'ander' in kl:
        ander+=res_chars[k]
    else:
        other+=res_chars[k]
        
print(chi, kolm, ander,other)

In [ ]:
picked_sum = collections.defaultdict(lambda: 0)
for tt in loader.tests.values():
    
    tt_id = '|'.join(reversed(tt.short_desc()))
    for vv in tt.variants.values():
        cfv = '|'.join([str(x) for x in vv.settings.values_tuple()])

        for ss in vv.sub_tests.values():
            tfv = '|'.join([str(x) for x in ss.params.values_tuple()])
            if len(ss.stats) == 0:
                logger.debug('Null statistics for test %s:%s:%s' % (tt_id, cfv, tfv))
                continue

            picked = dump_data.pick_one_statistic(ss.stats)
            picked_sum[picked.name] += 1

print(json.dumps(picked_sum, indent=2))       

In [ ]:
median